In [27]:
import os
import hashlib
import zipfile
import requests
import glob
import polars as pl
from datetime import datetime, timezone

icos_password = os.getenv("ICOS_PASS")
icos_user_email = os.getenv("ICOS_USER")

In [28]:
sites_meta = pl.read_csv("sites.csv", separator=";")

In [3]:
# calculate hashsum of a file
def hash_file(file_path:str) -> str:
    """Generates hashsum of a file
    Args:
        fname (str): _description_
    Returns:
        str: sha256 hashsum of the file
    """
    assert os.path.isfile(file_path)
    with open(file_path,"rb") as f:
        bytes = f.read() # read entire file as bytes
        return hashlib.sha256(bytes).hexdigest()

In [4]:
def zip_file(file_path: str) -> str:
    """Generates zipped version of a file
    Args:
        file_path (str): path to the file to be zipped
    Returns:
        str: path to the zipped file
    """
    assert os.path.isfile(file_path)
    zip_file_name = file_path + '.zip'
    with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Get the base name of the file (no directory structure)
        file_name_in_zip = os.path.basename(file_path)
        # Add the file to the zip, with no folder structure
        zipf.write(file_path, arcname=file_name_in_zip)
    return zip_file_name

In [15]:
datetime_format = '%Y-%m-%dT%H:%M:%SZ'
datetime.now(timezone.utc).strftime(datetime_format)

'2024-12-05T13:08:00Z'

In [ ]:
def upload_file_to_icos_cp(root_directory: str, fname: str,
                           site_id: str, sampling_height: str) -> bool:
    """Two step upload process to ICOS Carbon Portal. Register metadata package and upload file.

    Args:
        root_directory (str): root directory of the file
        fname (str): file name
        site_id (str): site id (must be available in the ICOS CP)
        sampling_height (str): variable sampling height

    Returns:
        Bool: True if successful
    """

    # zip the file and calculate hashsum
    try:
        zipped_filename = zip_file(root_directory + fname)
        hash_sum = hash_file(zipped_filename)
    except AssertionError:
        print(f"File {fname} not found")
        return False
    
    # STEP 1: register metadata package
    datetime_format = '%Y-%m-%dT%H:%M:%S.000Z'
    creation_date = datetime.now(timezone.utc).strftime(datetime_format) # date of the data creation
    creator_s = 'https://citymeta.icos-cp.eu/resources/people/Patrick_Aigner'
    contributor_s = ['https://citymeta.icos-cp.eu/resources/people/Jia_Chen', 'https://citymeta.icos-cp.eu/resources/people/KlausKurzinger']
    host_org_s = 'https://citymeta.icos-cp.eu/resources/organizations/TUM'
    data_object_specification = "https://citymeta.icos-cp.eu/resources/cpmeta/atmObsProduct"

    payload = {'submitterId': "TUM_MIDLOW",
            'hashSum': hash_sum,
            'fileName': fname + '.zip',
            'specificInfo': {'station': "https://citymeta.icos-cp.eu/resources/stations/Munich_" + site_id,
                                'samplingHeight': sampling_height,
                                'production': {
                                    "creator": creator_s,
                                    "contributors": contributor_s,
                                    "comment": "free text",
                                    "hostOrganization": host_org_s,
                                    "creationDate": creation_date}
                                },
            'objectSpecification': data_object_specification,
            'references': {'keywords':['CO2', 'Lowcost'],
                            'duplicateFilenameAllowed': False,
                            'autodeprecateSameFilenameObjects': True,
                            'partialUpload': False},
                'autodeprecateSameFilenameObjects': True,
                'duplicateFilenameAllowed': False,
                'partialUpload': False,
                }

    # Perform the POST request and save cookies to a file
    with requests.Session() as session:
        # authenticate
        auth = session.post("https://cpauth.icos-cp.eu/password/login",
                            data= {"mail": icos_user_email,
                                   "password": icos_password})
        
        r = session.post(url = 'https://citymeta.icos-cp.eu/upload', 
                         json=payload)  # Automatically sets the Content-Type to application/json)
        
        if r.status_code == 200:
            print(f"Register metadata package for site {site_id}: Successful")
            print(f'Response URL : {r.text}')
            print('Start uploading file')
            
            filepath = os.path.join(root_directory, fname +'.zip')
            with open(filepath, "rb") as file:
                rd = session.put(r.text, data=file)
            if rd.status_code == 200:
                print(f"File upload successful")
            else:
                print(f"File upload failed with status code {rd.status_code}")
                print(f"Response: {rd.text}")
                return False
        else: 
            print(f"Register metadata package: Failed with status code {r.status_code}")
            print(f"Response: {r.text}")
            return False
    
    return True

# Upload all files from folder

In [ ]:
path = os.path.join(os.getenv('DATA_DIRECTORY'),'processed','icos_cities_portal_processing','level_2','*.csv')
filenames = glob.glob(path)

for path in filenames:
    root_directory = os.path.basename(path)
    fname = os.path.basename(path)
    site_id = os.path.basename(path)[:4]
    
    if site_id == 'BLUT':
        sampling_height = os.path.basename(path)[5:7] + '.0'
    else:
        sampling_height = sites_meta.filter(pl.col("site") == site_id).select("height_of_building").item()
    
    print(root_directory,fname,site_id,sampling_height)
    #upload_file_to_icos_cp(root_directory,fname,site_id,sampling_height)

TAUR_munich_acropolis_2024.csv TAUR_munich_acropolis_2024.csv TAUR 17.89
BOGR_munich_acropolis_2024.csv BOGR_munich_acropolis_2024.csv BOGR 39.46
HARR_munich_acropolis_2024.csv HARR_munich_acropolis_2024.csv HARR 29.3
TUMR_munich_acropolis_2024.csv TUMR_munich_acropolis_2024.csv TUMR 31.06
RDIR_munich_acropolis_2024.csv RDIR_munich_acropolis_2024.csv RDIR 20.13
NPLR_munich_acropolis_2024.csv NPLR_munich_acropolis_2024.csv NPLR 31.43
SWMR_munich_acropolis_2024.csv SWMR_munich_acropolis_2024.csv SWMR 27.69
MAIR_munich_acropolis_2024.csv MAIR_munich_acropolis_2024.csv MAIR 15.08
DLRR_munich_acropolis_2024.csv DLRR_munich_acropolis_2024.csv DLRR 22.92
SCHR_munich_acropolis_2024.csv SCHR_munich_acropolis_2024.csv SCHR 17.14
BALR_munich_acropolis_2024.csv BALR_munich_acropolis_2024.csv BALR 18.17
FINR_munich_acropolis_2024.csv FINR_munich_acropolis_2024.csv FINR 15.21
FELR_munich_acropolis_2024.csv FELR_munich_acropolis_2024.csv FELR 15.73
BLUT_48_munich_acropolis_2024.csv BLUT_48_munich_acr